# Graph Neural Networks with new tracking dataset

Ming Fong

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import tensorflow as tf

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import uproot3 as ur
# import atlas_mpl_style as ampl
# ampl.use_atlas_style()

path_prefix = '/home/mfong/git/LCStudies/'
plotpath = path_prefix + 'classifier/Plots/'
modelpath = path_prefix + 'classifier/Models/'
# %config InlineBackend.figure_format = 'svg'

# metadata
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4]
len_eta = [128, 16, 8, 4, 4, 2]

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# set up GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.test.gpu_device_name())

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=24220)]) #in MB

print("Num GPUs Available: ", len(gpus))
tf.config.list_physical_devices()

/device:GPU:0
Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Input update notebook
https://github.com/janTOffermann/LCStudies/blob/master/classifier/input-update.ipynb

### Sanmay's code
https://github.com/sanmayphy/ML4PIONS_ATLAS/blob/main/modules/ML4Pions_Dataset.py

In [17]:
# TODO look at new data with positions and make edges relative distances

# pi0
# /clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/user.angerami.mc16_13TeV.900246.PG_singlepi0_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/user.angerami.24559740.OutputStream._000011.root
# pion
# /clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/user.angerami.mc16_13TeV.900247.PG_singlepion_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/user.angerami.24559744.OutputStream._000001.root

MLTreeData_dir = "/clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/"
pi0_dir = MLTreeData_dir + "user.angerami.mc16_13TeV.900246.PG_singlepi0_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/"
pion_dir = MLTreeData_dir + "user.angerami.mc16_13TeV.900247.PG_singlepion_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/"
pi0_root = pi0_dir + "user.angerami.24559740.OutputStream._000011.root"
pion_root = pion_dir + "user.angerami.24559744.OutputStream._000001.root"

pi0_file = ur.open(pi0_root)
pion_file = ur.open(pion_root)

# pi0_file["EventTree;1"].show()

# pi0_file["EventTree;1"].arrays("trackEta_EMB1", outputtype=pd.DataFrame, entrystop=100)

# pi0_file["EventTree;1"].arrays("trackPhi_EMB1", outputtype=pd.DataFrame, entrystop=100)

In [27]:
# pi0_file["EventTree;1"].pandas.df(flatten=False)
pi0_file["EventTree;1"].show()

runNumber                  (no streamer)              asdtype('>i4')
eventNumber                (no streamer)              asdtype('>i8')
lumiBlock                  (no streamer)              asdtype('>i4')
coreFlags                  (no streamer)              asdtype('>u4')
mcEventNumber              (no streamer)              asdtype('>i4')
mcChannelNumber            (no streamer)              asdtype('>i4')
mcEventWeight              (no streamer)              asdtype('>f4')
nTruthPart                 (no streamer)              asdtype('>i4')
G4PreCalo_n_EM             (no streamer)              asdtype('>i4')
G4PreCalo_E_EM             (no streamer)              asdtype('>f4')
G4PreCalo_n_Had            (no streamer)              asdtype('>i4')
G4PreCalo_E_Had            (no streamer)              asdtype('>f4')
truthVertexX               (no streamer)              asdtype('>f4')
truthVertexY               (no streamer)              asdtype('>f4')
truthVertexZ               (no str

In [20]:
pi0_file["CellGeo;1"].pandas.df()

cell_geo_ID  cell_geo_sampling  cell_geo_eta  cell_geo_phi  \
entry subentry                                                               
0     0           740294656                  6     -2.559710      0.053900   
      1           740294658                  6     -2.559648      0.151909   
      2           740294660                  6     -2.559603      0.249912   
      3           740294662                  6     -2.559574      0.347912   
      4           740294664                  6     -2.559562      0.445909   
...                     ...                ...           ...           ...   
      187645     1284491536                 15      0.958372     -0.049087   
      187646     1284491824                 17      1.058902     -0.049087   
      187647     1284492080                 17      1.159304     -0.049087   
      187648     1284492592                 17      1.309847     -0.049087   
      187649     1284493104                 17      1.510363     -0.049087   

                cell_geo_rPerp  cell_geo_deta  cell_geo_dphi  cell_geo_volume  \
entry subentry                                                                  
0     0             617.735962            0.1       0.098175     1.697610e+06   
      1             617.774719            0.1       0.098175     1.697610e+06   
      2             617.803223            0.1       0.098175     1.697610e+06   
      3             617.821167            0.1       0.098175     1.697610e+06   
      4             617.828552            0.1       0.098175     1.697610e+06   
...                        ...            ...            ...              ...   
      187645       3215.000000            0.1       0.098175     1.346147e+07   
      187646       2809.000000            0.1       0.098175     1.341334e+06   
      187647       2477.000000            0.1       0.098175     1.241210e+06   
      187648       2060.000000            0.2       0.098175     7.739876e+05   
      187649       1640.000000            0.2       0.098175     4.666501e+05   

                cell_geo_sigma  
entry subentry                  
0     0              49.457161  
      1              49.457161  
      2              49.457161  
      3              49.457161  
      4              49.457161  
...                        ...  
      187645         20.233513  
      187646         10.413343  
      187647         10.957963  
      187648         12.509910  
      187649         11.231500  

[187650 rows x 9 columns]